In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import lxml
import time
STATE_IDS = range(1, 37)
MAX_PAGES = 1000
BASE_URL = "https://www.myneta.info/LokSabha2024/index.php?action=show_constituencies&state_id={state_id}&page={page_num}"
OUTPUT_FILENAME = "lok_sabha_2024_candidates_full.csv"
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept-Language': 'en-US,en;q=0.5',
    'Referer': 'https://www.myneta.info/LokSabha2024/'
}
def extract_table_data(soup, state_name):
    main_table = soup.find('table', class_='w3-table w3-bordered')
    if not main_table:
        return None, None
    header_row = main_table.find('tr')
    if header_row:
        headers = [th.get_text(strip=True) for th in header_row.find_all('th')]
    else:
        headers = ['Sno.', 'Candidate', 'Party', 'Criminal Cases', 'Education', 'Age', 'Total Assets', 'Liabilities']
    headers = ["State"] + headers + ["Detail_Link"]
    data_rows = []
    for row in main_table.find_all('tr')[1:]:
        cols = row.find_all('td')
        if not cols:
            continue
        row_data = [state_name]
        detail_link = None
        for i, col in enumerate(cols):
            text_content = col.get_text(strip=True).replace('\n', ' ').strip()
            if i == 1:
                link_tag = col.find('a', href=True)
                if link_tag:
                    detail_link = link_tag['href']
            if col.find('img'):
                text_content = "Data in Image"
            elif 'Rs' in text_content and '~' in text_content:
                text_content = text_content.split('~')[0].strip()
            row_data.append(text_content)
        row_data.append(detail_link)
        data_rows.append(row_data)
    return data_rows, headers

def scrape_all_states(state_ids):
    all_data = []
    master_headers = None

    for state_id in state_ids:
        page_num = 1

        while page_num <= MAX_PAGES:
            url = BASE_URL.format(state_id=state_id, page_num=page_num)
            if page_num == 1 or page_num % 10 == 0 or page_num == MAX_PAGES:
                 print(f"-> Scraping State ID {state_id}, Page {page_num}...")

            try:
                response = requests.get(url, headers=HEADERS, timeout=20)
                response.raise_for_status()
                soup = BeautifulSoup(response.content, 'lxml')
                state_name_tag = soup.find('div', class_='w3-panel w3-leftbar w3-sand')
                state_name = "Unknown State"
                if state_name_tag:
                    state_name_text = state_name_tag.get_text(strip=True)
                    if '(' in state_name_text and ')' in state_name_text:
                        state_name = state_name_text.split('(')[-1].split(')')[0]

                current_data, headers = extract_table_data(soup, state_name)

                if current_data:
                    all_data.extend(current_data)
                    if master_headers is None:
                        master_headers = headers

                    page_num += 1
                    time.sleep(1)
                else:
                    print(f"   Finished State ID {state_id} ({state_name}) at Page {page_num - 1}. Stopping pagination.")
                    break

            except requests.exceptions.HTTPError as e:
                print(f"   HTTP Error for State ID {state_id} Page {page_num}: {e}. Stopping pagination.")
                break
            except Exception as e:
                print(f"   An unexpected error occurred for State ID {state_id} Page {page_num}: {e}. Stopping.")
                break
        time.sleep(5)

    return all_data, master_headers
start_time = time.time()
full_data, final_headers = scrape_all_states(STATE_IDS)
end_time = time.time()

if full_data:
    if final_headers:
        max_cols = max(len(row) for row in full_data)
        if len(final_headers) > max_cols:
            final_headers = final_headers[:max_cols]
        elif len(final_headers) < max_cols:
            final_headers.extend([f"Extra_Col_{i}" for i in range(len(final_headers), max_cols)])
        df = pd.DataFrame(full_data, columns=final_headers)
        df.to_csv(OUTPUT_FILENAME, index=False, encoding='utf-8')
    else:
        print("Data collected, but failed to determine headers for final CSV.")
else:
    print("\n Failed to collect any data.")

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import lxml
import time

URL = "https://www.myneta.info/LokSabha2024/index.php?action=recontestAssetsComparison"
OUTPUT_FILENAME = "lok_sabha_2024_asset_comparison.csv"

HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
}

def scrape_asset_comparison(url, headers):
    """
    Fetches the HTML, locates the asset comparison table, and extracts all data.
    """
    try:
        print(f"🌍 Fetching data from: {url}")
        response = requests.get(url, headers=headers, timeout=15)
        response.raise_for_status()

        soup = BeautifulSoup(response.content, 'lxml')

        main_table = soup.find('table', class_='w3-table w3-bordered')

        if not main_table:
            main_table = soup.find('table')
            if not main_table:
                print("Error: Could not find the main data table on the page.")
                return None, None

        headers = [th.get_text(strip=True) for th in main_table.find('tr').find_all('th')]

        data_rows = []
        for row in main_table.find_all('tr')[1:]:
            cols = row.find_all('td')
            if not cols:
                continue

            row_data = []

            for i, col in enumerate(cols):
                text_content = col.get_text(strip=True).replace('\n', ' ').strip()

                if 'Rs' in text_content and '~' in text_content:
                    text_content = text_content.split('~')[0].strip()

                if col.find('img'):
                    text_content = "Data in Image"

                if i == 1:
                    row_data.append(text_content)
                else:
                    row_data.append(text_content)

            data_rows.append(row_data)

        return data_rows, headers

    except requests.exceptions.RequestException as e:
        print(f"An error occurred during the web request: {e}")
        return None, None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None, None

# --- Main Execution ---
data, headers = scrape_asset_comparison(URL, HEADERS)

if data and headers:
    print(f"\nSuccessfully extracted {len(data)} data row(s).")

    num_cols_in_data = len(data[0]) if data else 0
    df = pd.DataFrame(data, columns=headers[:num_cols_in_data])
    df.dropna(how='all', inplace=True)

    df.to_csv(OUTPUT_FILENAME, index=False, encoding='utf-8')
    print(f"💾 Data saved successfully to **{OUTPUT_FILENAME}**")
    print("\n--- DataFrame Head ---")
    print(df.head().to_markdown(index=False))
else:
    print("\n Scraping failed. No data to process.")
